# Etude de cas de l'utilisation des IA Génératives (LLMs) dans l'enseignement supérieur de la science du numérique.

---

## Préambule

Les IA génératives telles que ChatGPT, DeepSeek, ou Claude Sonnet sont indéniablement devenues ces dernières années des outils très prominents dans le quotidien numérique des étudiants. Étant des outils très puissants, il est essentiel de savoir s'en servir convenablement pour en tirer le maximum. Cela demande en premier lieu d'*au moins* comprendre dans les très (très) grandes lignes leur principe de fonctionnement et ce que la technologie implique, mais aussi leurs potentiel et limites actuelles. Cela évitera notamment ce genre de remarques:

<div>
    <center>
    <img src="img/illustrative_tweet.png" width="500"/>
</div>

## Principe de l'IA Générative

Pour faire très (très) simple en omettant toutes les maths derrière:

&rarr; un **modèle de langage** comme **GPT 4.1** (≠ ChatGPT!) va, à partir d'un texte en entrée (un contexte en fait), découper ce texte en morceaux/~mots appelés *tokens* et leur attribuer un vecteur dans un espace de très grande dimension (beaucoup d'axes). 

&rarr; Ce vecteur déterminera d'une certaine manière le sens "*sémantique*" d'un token, et ainsi les autres mots de sens similaires auront des vecteurs proches dans l'espace. Les valeurs qui caractérisent ces vecteurs sont choisies aléatoirement puis apprises lors de l'**apprentissage** du modèle; une étape antérieure où on apprend au modèle à comparer et donc catégoriser les entrées qu'on lui donne.

Plus bas il y a par exemple un code en Python que vous pouvez exécuter (pas besoin de chercher à le comprendre, il faut juste cliquer sur le bouton latéral gauche pour démarrer la zone de code) afin d'observer l'angle entre des vecteurs de tokens:

In [ ]:
## On installe une librairie qui permet de télécharger de larges modèles utilisés dans le traîtement du 
# langage naturel

#à ne lancer qu'une fois et à installer sur Python <=3.12 !
pip install gensim

On utilise le modèle ***glove-wiki-gigaword-50***, qui n'est pas un LLM mais un *Word Embedding Model*. Il s'agit dans tous les cas exactement de ce qu'on a décrit dans le paragraphe précédent: C'est un modèle qui capture les relations sémantiques entre les mots et peut les représenter dans des vecteurs spaciaux qui symbolisent leur proximité sémantique.

On importe *gensim* et on télécharge le modèle:

In [2]:
import gensim.downloader # On importe la librairie qui nous permet de télécharger des modèles de 
# plongement de mots

model = gensim.downloader.load("glove-wiki-gigaword-50")

On va observer la proximité sémantique des tokens en regardant l'angle que font leur vecteur:

In [3]:
# On va utiliser les fonctions intégrées à Numpy pour traiter nos vecteurs
import numpy as np

## Fonctions utilitaires
def normaliser(vecteur):
    """ Normalise le vecteur en argument.
    """
    return vecteur / np.linalg.norm(vecteur)

def angle(v1, v2):
    """Retourne l'angle fait par les vecteurs donnés en argument.
    """
    v1_u = normaliser(v1)
    v2_u = normaliser(v2)
    res = np.dot(v1_u, v2_u)
    res = np.clip(res, -1.0, 1.0)
    res = np.arccos(res)
    res *= 180/np.pi
    return round(res, 3)

## Affichage des vecteurs

#Exemple avec les vecteurs des mots "tour" et "bâtiment" 
vecteur_jeu = model["tower"] 

vecteur_player = model["building"]

print(f"Le vecteur normalisé du mot 'tower' est:\n {normaliser(vecteur_jeu)}\n")
print(f"Le vecteur normalisé du mot 'building' est:\n {normaliser(vecteur_player)}\n")
print(f"L'angle entre les deux vecteurs en degrés est environ {angle(vecteur_jeu, vecteur_player)}")



Le vecteur normalisé du mot 'tower' est:
 [ 2.32295305e-01  2.39117995e-01  1.50941327e-01 -1.19652124e-02
  1.02237053e-01 -1.42626569e-01 -6.50604963e-02 -9.18937027e-02
 -9.26488563e-02 -1.52530596e-01 -6.78442419e-02 -5.05627971e-03
 -1.01615526e-01  1.29110754e-01 -1.68155968e-01  1.69180378e-01
 -5.00080697e-02  6.56374916e-02 -2.24784255e-01 -4.31934837e-03
  1.39120072e-01 -7.96452612e-02 -3.31821531e-01 -1.02224916e-01
 -3.37773636e-02 -1.36957854e-01 -6.43762052e-02  1.79177538e-01
 -6.38781721e-03 -3.15989628e-02  4.00959432e-01 -2.40312472e-01
  1.68728903e-01 -3.71887088e-02 -5.40368967e-02  2.35230885e-02
  2.23164648e-01 -7.27557614e-02  5.06437756e-03 -8.22265446e-03
  6.21147975e-02 -8.31598490e-02  1.71247441e-02  4.55014519e-02
 -1.03160247e-01  1.33252963e-01 -2.51690354e-02 -2.84224659e-01
  3.27529488e-05 -1.06346868e-01]

Le vecteur normalisé du mot 'building' est:
 [ 0.17743999  0.09454948  0.07480241 -0.08101078  0.09619492 -0.07971676
 -0.16223752 -0.13204496 

Voici un autre exemple avec des mots sans grande liaison sémantique:

In [4]:

vecteur_jeu = model["printer"] # veccteur pour imprimante

vecteur_player = model["platypus"] # vecteur pour ornythorinque

# print(f"Le vecteur normalisé du mot 'game' est:\n {normaliser(vecteur_jeu)}\n")
# print(f"Le vecteur normalisé du mot 'player' est:\n {normaliser(vecteur_player)}\n")
print(f"L'angle entre les deux vecteurs en degrés est environ {angle(vecteur_jeu, vecteur_player)}")


L'angle entre les deux vecteurs en degrés est environ 105.659



&rarr; Ces tokens échangent ensuite entre eux pour décider quels éléments du contexte influent sur la **sémantique appropriée** de certains mots de la phrase. Par exemple, quand on dit "graver en mémoire", c'est un verbe 'graver' qui diffère de celui dans "graver dans de la pierre". On peut appeler cette étape "*bloc d'Attention*".

&rarr; Ensuite, ces vecteurs passent par une étape au nom barbare de *perceptron multi-couches* ... le procédé mathématique exact est un peu compliqué à vulgariser donc on va juste dire que c'est une manière pour le modèle de catégoriser et mettre à jour chaque vecteur de token, comme s'il lui posait à la suite pleins de questions et qu'il prenait en compte les réponses pour affiner le vecteur.

&rarr; La suite est une répétition d'alternances entre **blocs d'attention** et **perceptrons multi-couches**, jusqu'à la dernière itération où le dernier vecteur de la séquence sera donc le résultat combiné de tout ce passage.

&rarr; En opérant sur ce vecteur, on obtient une **distribution de probabilités** de tous les *tokens* possibles en sortie (donc tous les mots/segments de texte qui peuvent se placer linguistiquement dans la continuité de notre texte initial). En gros, on obtient une liste des mots qui pourront **probablement** le mieux compléter la suite du texte donné.

Mais tout ¢a, c'est pour obtenir **un** mot. Ainsi, on peut recommencer une fois de plus en ajoutant notre mot nouvellement obtenu dans le contexte de l'itération suivante afin d'obtenir la suite. Et on peut recommencer encore et encore...

Enfin, pour obtenir un ***Chatbot*** (pour faire simple, ChatGPT et équivalent) depuis ce modèle, en fait c'est tout bête voire assez marrant: On appose, en préfixe du prompt de l'utilisateur, un *contexte caché* où on précise que ce qui suit est une discussion entre une IA et son utilisateur. Donc, le prompt de l'utilisateur sera donné à l'IA sous la forme du début d'un dialogue. De là, le chatbot va en fait prédire mot à mot la réponse la plus probable que ferait un assistant IA dans le dialogue.

SUITE A COMPLETER SUR LE PRE-TRAINING / FINE-TUNING. C'est pas fini...

- ***TODO***

mécanisme d'attention

embedding / unembedding

context size et autre

Générative ≠ Créative -> Dépend du data set 
d'apprentissage; Il établit ses pondérations par rapport à ce qu'il a appris.

Biais d'apprentissage ---> biais de confirmation venant du fine-tuning, et biais du dataset.

## Limites des LLMs



## Observer les pondérations que l'IA donne aux tokens

Pour observer la pondération que l'IA attribue aux tokens pour comprendre le contexte, on peut utiliser l'outil [*interpreto*](https://pypi.org/project/interpreto/), qui a pour ambition de rendre les LLMs plus transparents et de trouver une manière de mesurer leur fiabilité.

In [ ]:
pip install interpreto